# Import Modules

In [198]:
import os
import random

# Create Classes

In [199]:
class FullTrackList():
    def __init__(self, filename):
        self.filename(filename)
        
    def filename(self, filename):
        """Setter used to set the filename and validate it, also gets tracks from the file and puts them into a list"""

        # Tests if the CSV file supplied is valid and exists
        if type(filename) == str and filename.endswith(".csv") and os.path.isfile(filename):
            self._filename = filename
            self._tracks = []
            with open(self._filename, "r") as fh:
                for line in fh.readlines():
                    split_line = line.split(",")
                    self._tracks.append(Track(id=split_line[0], artist=split_line[1], title=split_line[2], time=TimeMS(split_line[3])))
            
        else:
            raise TypeError("Error, invalid file selected")
        
    def get_filename(self):
        """Getter, used to get the filename"""
        return self._filename

    def get_tracks(self):
        """Getter, used to get the list of tracks in the tracklist"""
        return self._tracks

    def __repr__(self):
        the_rep = ""
        for track in self._tracks:
            the_rep += str(track) + "\n"
        return the_rep


class Playlist():
    """Class representing a user playlist"""
    def __init__(self, track_ids, tracklist):
        self._track_ids = track_ids

        # Tracklist will be an object from FullTrackList Class
        if isinstance(tracklist, FullTrackList):
            self._tracklist = tracklist
        
        else:
            raise TypeError("Error: Incorrect type for tracklist. Should be a member of the FullTrackList class")
            
        # Initialise the time attribute = 0, add to this value as we add tracks
        self._time = 0

        # Tracks that correspond to the track IDs provided by the user will be added to the playlist
        self.tracks = []
        for track in self._tracklist.get_tracks():
            if track.id in self._track_ids:
                self.tracks.append(track)
                self._time += int(track.time.in_seconds())

    def show_playlist(self):
        """Shows each track in the playlist and the total playing time"""
        # Create a Time object using self._time, which is the total time in seconds for each track
        # Then, represent this Time object in minutes to get the total track time in minutes
        print("Playing Time:", TimeMS(str(self._time)))
        for track in self.tracks:
            print("\t" + str(track.id) + "\t" + str(track.artist) + ":", track.title)

    def add_track(self, track_id):
        """Adds a track to the playlist based on the track ID provided by the user
        
        Adds nothing if the user provides an invalid track ID
        """
        for track in self._tracklist.get_tracks():
            if track.id == track_id:
                self.tracks.append(track)
                self._time += int(track.time.in_seconds())

    def remove_track(self, track_id):
        """Removes a track from the playlist based on the track ID provided by the user
        
        Removes nothing if the user provides an invalid track ID
        """
        for track in self.tracks:
            if track.id == track_id:
                self.tracks.remove(track)
                self._time -= int(track.time.in_seconds())

    def shuffle_playlist(self):
        """Shuffles the playlist"""
        # using Fisher–Yates shuffle Algorithm to shuffle a list
        for i in range(len(self.tracks)-1, 0, -1):
            
            # Pick a random index from 0 to i
            j = random.randint(0, i)
        
            # Swap arr[i] with the element at random index
            self.tracks[i], self.tracks[j] = self.tracks[j], self.tracks[i]

    def sort_playlist(self):
        """Sorts the playlist based on the artist name and song title"""
        self.tracks.sort(key=lambda x: (x.artist, x.title))

    def __repr__(self):
        """Provides a representation for an object in the Playlist class"""
        the_rep = "Playing Time: " + str(TimeMS(str(self._time)))
        for track in self.tracks:
            the_rep += "\n\t " + str(track.id) + "\t " + str(track.artist) + " : " + track.title
        return the_rep


class Track():
    def __init__(self, id, artist, title, time):
        self.id = id
        self.artist = artist
        self.title = title
        self.time = time

    def __lt__(self, other):
        return int(self.time.in_seconds()) < int(other.time.in_seconds())

    def __repr__(self):
        """Representation of a Track object"""
        the_rep = "Track ID: " + self.id + "\nArtist: " + self.artist + "\nTitle: " + self.title + "\nTime: " + str(self.time)
        return the_rep


class TimeMS():
    def __init__(self, time):
        if ":" in str(time):
            self.min_to_sec(time)
        
        else:
            self.sec_to_min(time)

    def min_to_sec(self, time):
        """First represent time as minutes, then convert this value to seconds for the seconds representation
        
        Assigns time to the minutes representation
        Splits the minutes up into it's two components, the minute component and the second component (EG: 21 minutes and 37 seconds)
        Converts the minutes component to seconds by multiplying by 60, then adds the seconds component to get seconds representation
        """
        self._full_mins = time
        self._full_mins_split = time.split(":")
        self._min_split, self._sec_split = self._full_mins_split[0], self._full_mins_split[1]
        self._full_seconds = (int(self._min_split) * 60) + int(self._sec_split)

    def sec_to_min(self, time):
        """First represent the time as seconds, then convert this value to minutes for the minutes representation
        
        Assigns time to the seconds representation
        Creates two variables, one representing the minutes component (initially 0) and one representing the seconds component (initially the seconds representation)
        Since the seconds cannot be greater than or equal to 60 in the minutes representation, loop through the seconds component while it is greater than or equal to 60
        On each loop iteration, subtracts 60 from the seconds component and increments the minutes component by 1
        Uses an if statement to allow for the situation where the seconds component is a single digit number by prefixing zero in front of the seconds component
        Once both components are correct, concatenate them and assign this concatenation to the minutes representation
        """
        self._full_seconds = time
        self._sec_split = int(self._full_seconds)
        self._min_split = 0
        while self._sec_split >= 60:
            self._sec_split -= 60
            self._min_split += 1

        if len(str(self._sec_split)) == 1:
            self._sec_split = "0" + str(self._sec_split)
            
        else:
            self._sec_split = str(self._sec_split)

        self._full_mins = str(self._min_split) + ":" + self._sec_split

    def in_seconds(self):
        """Gets the seconds representation"""
        return self._full_seconds

    def __lt__(self, other):
        """Determines the behaviour of the < operator when dealing with members of the TimeMS class"""
        return self._full_seconds < other._full_seconds

    def __repr__(self):
        """Default representation of a TimeMS object will be in minutes"""
        return self._full_mins



# Testing

In [200]:
tlist = FullTrackList("music.csv")
print(tlist.get_filename())
print(tlist.get_tracks())

music.csv
[Track ID: 1
Artist: Taylor Swift
Title: Everything Has Changed
Time: 4:12
, Track ID: 2
Artist: Mumford and Sons
Title: Little Lion Man
Time: 4:08
, Track ID: 3
Artist: Hozier
Title: Sedated
Time: 3:30
, Track ID: 4
Artist: Mumford and Sons
Title: Babel
Time: 4:05
, Track ID: 5
Artist: Taylor Swift
Title: I Knew You Were Trouble
Time: 3:40
, Track ID: 6
Artist: Taylor Swift
Title: We Are Never Ever Getting Back Together
Time: 3:16
, Track ID: 7
Artist: Hozier
Title: Jackie and Wilson
Time: 3:44
, Track ID: 8
Artist: Hozier
Title: Take Me To Church
Time: 4:02
, Track ID: 9
Artist: Lily Allen
Title: Not Fair
Time: 3:50
, Track ID: 10
Artist: Hozier
Title: Angel of Small Death & The Codeine Scene
Time: 3:34
, Track ID: 11
Artist: Post Malone
Title: Better Now
Time: 3:26
, Track ID: 12
Artist: Miley Cyrus
Title: Jolene
Time: 2:29
, Track ID: 13
Artist: Florence + The Machine
Title: Dog Days are Over
Time: 3:44
, Track ID: 14
Artist: Rufus Wainwright
Title: April Fools
Time: 4:03

In [201]:
pl = Playlist(["1", "2", "3"], tlist)
pl.show_playlist()

Playing Time: 11:50
	1	Taylor Swift: Everything Has Changed
	2	Mumford and Sons: Little Lion Man
	3	Hozier: Sedated


In [202]:
pl.add_track("7")
pl.add_track("8")
pl.add_track("12")
pl.show_playlist()

Playing Time: 22:05
	1	Taylor Swift: Everything Has Changed
	2	Mumford and Sons: Little Lion Man
	3	Hozier: Sedated
	7	Hozier: Jackie and Wilson
	8	Hozier: Take Me To Church
	12	Miley Cyrus: Jolene


In [203]:
pl.add_track("13")
pl.remove_track("1")
pl.shuffle_playlist()
pl.show_playlist()

Playing Time: 21:37
	12	Miley Cyrus: Jolene
	3	Hozier: Sedated
	2	Mumford and Sons: Little Lion Man
	8	Hozier: Take Me To Church
	7	Hozier: Jackie and Wilson
	13	Florence + The Machine: Dog Days are Over


In [204]:
pl.sort_playlist()
pl.show_playlist()

Playing Time: 21:37
	13	Florence + The Machine: Dog Days are Over
	7	Hozier: Jackie and Wilson
	3	Hozier: Sedated
	8	Hozier: Take Me To Church
	12	Miley Cyrus: Jolene
	2	Mumford and Sons: Little Lion Man


In [205]:
pl

Playing Time: 21:37
	 13	 Florence + The Machine : Dog Days are Over
	 7	 Hozier : Jackie and Wilson
	 3	 Hozier : Sedated
	 8	 Hozier : Take Me To Church
	 12	 Miley Cyrus : Jolene
	 2	 Mumford and Sons : Little Lion Man

In [206]:
tlist

Track ID: 1
Artist: Taylor Swift
Title: Everything Has Changed
Time: 4:12

Track ID: 2
Artist: Mumford and Sons
Title: Little Lion Man
Time: 4:08

Track ID: 3
Artist: Hozier
Title: Sedated
Time: 3:30

Track ID: 4
Artist: Mumford and Sons
Title: Babel
Time: 4:05

Track ID: 5
Artist: Taylor Swift
Title: I Knew You Were Trouble
Time: 3:40

Track ID: 6
Artist: Taylor Swift
Title: We Are Never Ever Getting Back Together
Time: 3:16

Track ID: 7
Artist: Hozier
Title: Jackie and Wilson
Time: 3:44

Track ID: 8
Artist: Hozier
Title: Take Me To Church
Time: 4:02

Track ID: 9
Artist: Lily Allen
Title: Not Fair
Time: 3:50

Track ID: 10
Artist: Hozier
Title: Angel of Small Death & The Codeine Scene
Time: 3:34

Track ID: 11
Artist: Post Malone
Title: Better Now
Time: 3:26

Track ID: 12
Artist: Miley Cyrus
Title: Jolene
Time: 2:29

Track ID: 13
Artist: Florence + The Machine
Title: Dog Days are Over
Time: 3:44

Track ID: 14
Artist: Rufus Wainwright
Title: April Fools
Time: 4:03

Track ID: 15
Artist: P